In [31]:
import pandas as pd
import numpy as np

In [2]:
PATH = 'data/'

In [3]:
pwd

'/notebooks/NLP'

In [45]:
df = pd.read_csv(f'{PATH}train.csv')
df.describe(include='object') # object -> only string columns

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [38]:
eval_df = pd.read_csv(f'{PATH}test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [7]:
df['input'] = 'TEXT1: ' + df['context'] + '; TEXT2: ' + df['target'] + '; ANC1: ' + df['anchor'] 

In [8]:
df['input'].head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [9]:
# import huggingface dataset
from datasets import Dataset, DatasetDict

In [10]:
ds = Dataset.from_pandas(df)

In [11]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [12]:
model_nm = 'microsoft/deberta-v3-small'

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
tokz.tokenize("Tom is going home now! I'm happy")

['▁Tom', '▁is', '▁going', '▁home', '▁now', '!', '▁I', "'", 'm', '▁happy']

In [15]:
tokz.tokenize("A platypus is an ornithorhynchus anatinus.")

['▁A',
 '▁platypus',
 '▁is',
 '▁an',
 '▁or',
 'ni',
 'tho',
 'rhynch',
 'us',
 '▁an',
 'at',
 'inus',
 '.']

In [16]:
def tok_func(x):
    return tokz(x['input'])

In [17]:
tok_ds = ds.map(tok_func, batched=True)

Parameter 'function'=<function tok_func at 0x7fc905cc6c10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/37 [00:00<?, ?ba/s]

In [18]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [19]:
row

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 'input_ids': [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokz.vocab['▁of']

265

In [21]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [22]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [39]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
def corr(x,y):
    return np.corrcoef(x,y)[0][1]

In [24]:
def corr_d(eval_pred):
    return {'person':corr(*eval_pred)}

In [25]:
from transformers import TrainingArguments, Trainer

In [26]:
bs = 128
epochs = 4

In [27]:
lr = 8e-5

In [28]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
                        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_gpu_eval_batch_size=bs*2,
                        num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                 tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [34]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, context, id, target, anchor. If input, context, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 856


Epoch,Training Loss,Validation Loss,Person
1,No log,0.022293,0.833160
2,No log,0.022767,0.835525
3,0.014000,0.021645,0.836093
4,0.014000,0.021725,0.831246


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, context, id, target, anchor. If input, context, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input

TrainOutput(global_step=856, training_loss=0.013395316132875246, metrics={'train_runtime': 356.0247, 'train_samples_per_second': 307.327, 'train_steps_per_second': 2.404, 'total_flos': 1074351445395960.0, 'train_loss': 0.013395316132875246, 'epoch': 4.0})

In [40]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, context, id, target, anchor. If input, context, id, target, anchor are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


array([[ 0.49438477],
       [ 0.59570312],
       [ 0.63867188],
       [ 0.30883789],
       [-0.00110531],
       [ 0.52148438],
       [ 0.48974609],
       [ 0.01122284],
       [ 0.25341797],
       [ 1.01660156],
       [ 0.22705078],
       [ 0.26220703],
       [ 0.75537109],
       [ 0.83789062],
       [ 0.74609375],
       [ 0.45483398],
       [ 0.35229492],
       [ 0.01364136],
       [ 0.61572266],
       [ 0.37353516],
       [ 0.49414062],
       [ 0.22717285],
       [ 0.26464844],
       [ 0.24328613],
       [ 0.56835938],
       [ 0.02981567],
       [-0.00384331],
       [ 0.01403046],
       [ 0.01806641],
       [ 0.78466797],
       [ 0.30224609],
       [ 0.04428101],
       [ 0.72167969],
       [ 0.49536133],
       [ 0.47094727],
       [ 0.22607422]])

In [41]:
preds = np.clip(preds, 0, 1)

In [44]:
import datasets

submissions = datasets.Dataset.from_dict({
    'id':eval_ds['id'],
    'score':preds
})

submissions.to_csv('/submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1065